# Assignment 5   
## Name: Akram Shaik   
## Z-ID: Z1857655

In [0]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [155]:
# uploading input data locally
'''from google.colab import files
uploaded = files.upload()
import io
reddit_train = pd.read_csv(io.BytesIO(uploaded['reddit_200k_train.csv']), encoding='unicode_escape')
reddit_test = pd.read_csv(io.BytesIO(uploaded['reddit_200k_test.csv']), encoding='unicode_escape')
'''

"from google.colab import files\nuploaded = files.upload()\nimport io\nreddit_train = pd.read_csv(io.BytesIO(uploaded['reddit_200k_train.csv']), encoding='unicode_escape')\nreddit_test = pd.read_csv(io.BytesIO(uploaded['reddit_200k_test.csv']), encoding='unicode_escape')\n"

In [156]:
#uploading train and test data from github link
df = pd.read_csv("https://raw.githubusercontent.com/akramshaik2326/sample/master/reddit_200k_train.csv", encoding='latin1')
df1 = pd.read_csv("https://raw.githubusercontent.com/akramshaik2326/sample/master/reddit_200k_test.csv", encoding='latin1')
#printing shapes of both datasets
print(df.shape, df1.shape)

(167529, 8) (55843, 8)


In [157]:
#X_train = pd.DataFrame(df['body'])
#y_train = pd.DataFrame(df['REMOVED'])
#X_test = pd.DataFrame(df1['body'])
#y_test = pd.DataFrame(df1['REMOVED'])

# extracting required columns from the dataset in labels only body columns is required and target is the removed column
X_train, y_train = df.body, df.REMOVED
X_test, y_test = df1.body, df1.REMOVED

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(167529,) (167529,)
(55843,) (55843,)


In [158]:
X_test.dtype

dtype('O')

In [159]:
y_train.head()

0    False
1     True
2     True
3    False
4    False
Name: REMOVED, dtype: bool

In [160]:
#printing content of X_train whiuch contains tweets
X_train.head()

0    I've always been taught it emerged from the ea...
1    As an ECE, my first feeling as "HEY THAT'S NOT...
2    Monday: Drug companies stock dives on good new...
3    i learned that all hybrids are unfertile i won...
4    Well i was wanting to get wasted tonight.  Not...
Name: body, dtype: object

In [163]:
# applying label encoder on target dataset as the values are in text form
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)
print(y_train_enc.shape, y_train_enc)
print(y_test_enc.shape, y_test_enc)

(167529,) [0 1 1 ... 0 1 0]
(55843,) [1 0 0 ... 1 1 0]


In [0]:
# importing countvectorizer from sklearn
from sklearn.feature_extraction.text import CountVectorizer

#creating object of countvectorizer with pareameter stopwords english as it will remove all stopwords from tweets
vect = CountVectorizer(stop_words='english')

#fitting transforming the train and test data to create bag of words model
bow_train = vect.fit_transform(X_train)
bow_test  = vect.transform(X_test)

In [165]:
#printing shapes of bag of words of train and test data
print(bow_test.shape, bow_train.shape)

(55843, 113494) (167529, 113494)


In [0]:
# importing logistic regression to apply classification 
from sklearn.linear_model import LogisticRegression
# using gridsearchcv to tune the classificatiuon model with right value of regularization
from sklearn.model_selection import GridSearchCV

# Assignment list of c values to find the one that gives best results
# not large range of values are added in C and not many parmeters have been given as the dataset is huge it will take a large ammount of time to execute
grid = {"C" : [0.001,0.01,0.1, 0.5, 1] }

#creating model instance
logreg = LogisticRegression(class_weight='balanced', random_state=1)
#implementing gridsearch with 10 folds
logreg_cv = GridSearchCV( logreg, grid, cv = 10 )

In [167]:
# fitting bag of words on model
logreg_cv.fit(bow_train, y_train_enc)
#printing the best parameters and accuracy
print("tuned hpyerparameters : (best parameters) ", logreg_cv.best_params_)
print(" Train data accuracy : ", logreg_cv.best_score_)

tuned hpyerparameters : (best parameters)  {'C': 0.1}
 Train data accuracy :  0.6707853579917347


In [168]:
# printing accuracy of model on test data
print("Test data accuracy : ", logreg_cv.score(bow_test, y_test_enc))

Test data accuracy :  0.6719373959135433


In [169]:
# finding or predicting target y values on test data
y_pred = logreg_cv.predict( bow_test)

# finding confusion matrix and classification report of the model's prediction based on actual and predicted values
from sklearn.metrics import classification_report, confusion_matrix
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y_pred ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y_pred ) )

# finding average f1 score of the model
from sklearn.metrics import f1_score
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y_pred ) ) )

 Confusion Matrix : 
 [[20464 14101]
 [ 4219 17059]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.83      0.59      0.69     34565
           1       0.55      0.80      0.65     21278

    accuracy                           0.67     55843
   macro avg       0.69      0.70      0.67     55843
weighted avg       0.72      0.67      0.68     55843

f1_score of logistic regression on scaled data : 0.651


# ***Question: 2***

In [0]:
# using vect4 instance which is based on value of min df that is 4,on char analyzer with ngram range (1,1)
# different min df values were evaluated but 4 gave best results
vect4 = CountVectorizer(min_df=4, stop_words='english', token_pattern=r"\b\w[\w’]+\b", ngram_range=(1, 1), analyzer='char_wb')
bow_train4 = vect4.fit_transform(X_train)
bow_test4 = vect4.transform(X_test)

In [0]:
#applyig tfidf on the bow model
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.pipeline import make_pipeline
X_train_tfidf = TfidfTransformer().fit_transform(bow_train4)

In [0]:
X_test_tfidf = TfidfTransformer().fit_transform(bow_test4)

In [174]:
logreg_cv.fit(X_train_tfidf, y_train_enc)
#printing the best parameters and accuracy
print("tuned hpyerparameters : (best parameters) ", logreg_cv.best_params_)
print(" Train data accuracy : ", logreg_cv.best_score_)

# printing accuracy of model on test data
print("Test data accuracy : ", logreg_cv.score(X_test_tfidf, y_test_enc))

tuned hpyerparameters : (best parameters)  {'C': 1}
 Train data accuracy :  0.6602558749260492
Test data accuracy :  0.6608885625772254


In [175]:
# finding or predicting target y values on test data
y_pred = logreg_cv.predict( X_test_tfidf)

# finding confusion matrix and classification report of the model's prediction based on actual and predicted values
from sklearn.metrics import classification_report, confusion_matrix
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y_pred ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y_pred ) )

# finding average f1 score of the model
from sklearn.metrics import f1_score
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y_pred ) ) )

 Confusion Matrix : 
 [[22721 11844]
 [ 7093 14185]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.76      0.66      0.71     34565
           1       0.54      0.67      0.60     21278

    accuracy                           0.66     55843
   macro avg       0.65      0.66      0.65     55843
weighted avg       0.68      0.66      0.67     55843

f1_score of logistic regression on scaled data : 0.600


In [0]:
#vect4 = CountVectorizer(min_df=4, stop_words='english', token_pattern=r"\b\w[\w’]+\b", ngram_range=(1, 3), analyzer='char_wb')
# In char analyzer with ngram range (1,3) execution time was very high and the results were not better than ngram range (1,2)
# so we finally consider comparing word and char analyer with ngram range in char as (1,2)  
vect4 = CountVectorizer(min_df=4, stop_words='english', token_pattern=r"\b\w[\w’]+\b", ngram_range=(1, 2), analyzer='char_wb')
bow_train4 = vect4.fit_transform(X_train)
bow_test4 = vect4.transform(X_test)

In [0]:
# importing tfidf transformer from sklearn
from sklearn.feature_extraction.text import TfidfTransformer
# fitting and transformig bag of words on tfidf 
X_train_tfidf = TfidfTransformer().fit_transform(bow_train4)

In [0]:
# fitting and transforming test data bag of words on tfidf
X_test_tfidf = TfidfTransformer().fit_transform(bow_test4)

In [179]:
# fitting tfidf applied vectors on logistic regression
logreg_cv.fit(X_train_tfidf, y_train_enc)
#printing the best parameters and accuracy
print("tuned hpyerparameters : (best parameters) of tfidf applied BoW ", logreg_cv.best_params_)
print(" Train data accuracy of tfidf : ", logreg_cv.best_score_)

# printing accuracy of model on test data
print("Test data accuracy : ", logreg_cv.score(X_test_tfidf, y_test_enc))

tuned hpyerparameters : (best parameters) of tfidf applied BoW  {'C': 1}
 Train data accuracy of tfidf :  0.6919041125565559
Test data accuracy :  0.6933724907329477


In [180]:
# finding or predicting target y values on test data
y_pred = logreg_cv.predict( X_test_tfidf)

# finding confusion matrix and classification report of the model's prediction based on actual and predicted values
from sklearn.metrics import classification_report, confusion_matrix
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y_pred ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y_pred ) )

# finding average f1 score of the model
from sklearn.metrics import f1_score
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y_pred ) ) )

 Confusion Matrix : 
 [[24011 10554]
 [ 6569 14709]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.69      0.74     34565
           1       0.58      0.69      0.63     21278

    accuracy                           0.69     55843
   macro avg       0.68      0.69      0.68     55843
weighted avg       0.71      0.69      0.70     55843

f1_score of logistic regression on scaled data : 0.632


# Word analyzer   

### First analyzing on min df -2 

In [0]:
vect2 = CountVectorizer(min_df=2, stop_words='english', token_pattern=r"\b\w[\w’]+\b")
bow_train2 = vect2.fit_transform(X_train)
bow_test2 = vect2.transform(X_test)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.pipeline import make_pipeline
X_train_tfidf = TfidfTransformer().fit_transform(bow_train2)
X_test_tfidf = TfidfTransformer().fit_transform(bow_test2)

In [183]:
logreg_cv.fit(X_train_tfidf, y_train)
#printing the best parameters and accuracy
print("tuned hpyerparameters : (best parameters) ", logreg_cv.best_params_)
print(" Train data accuracy : ", logreg_cv.best_score_)

# printing accuracy of model on test data
print("Test data accuracy : ", logreg_cv.score(X_test_tfidf, y_test_enc))

tuned hpyerparameters : (best parameters)  {'C': 1}
 Train data accuracy :  0.6872422053618987
Test data accuracy :  0.6893254302240209


In [184]:
# finding or predicting target y values on test data
y_pred = logreg_cv.predict( X_test_tfidf)

# finding confusion matrix and classification report of the model's prediction based on actual and predicted values
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y_pred ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y_pred ) )

# finding average f1 score of the model
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y_pred ) ) )

 Confusion Matrix : 
 [[22721 11844]
 [ 5505 15773]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.80      0.66      0.72     34565
           1       0.57      0.74      0.65     21278

    accuracy                           0.69     55843
   macro avg       0.69      0.70      0.68     55843
weighted avg       0.72      0.69      0.69     55843

f1_score of logistic regression on scaled data : 0.645


### Second analyzing with min df -4 and applying stemming with ngram range 2

In [0]:
# applying stemming on train and test data to check if this helps increasing the accuracy
X_train1 = pd.DataFrame(df['body'])
X_test1 = pd.DataFrame(df1['body'])
# importing porterstemmer from nltk
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

#applying stemmer on train data
for i in X_train1:
    i = [porter.stem(word) for word in i]
#applying stemmer on test data
for i in X_test1:
    i = [porter.stem(word) for word in i]

X_train1 = X_train1.body
X_test1 = X_test1.body

In [186]:
print(X_train1.shape, X_test1.shape)

(167529,) (55843,)


In [0]:
vect4 = CountVectorizer(min_df=4, stop_words='english', token_pattern=r"\b\w[\w’]+\b", ngram_range=(1, 2))
bow_train4 = vect4.fit_transform(X_train1)
bow_test4 = vect4.transform(X_test1)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.pipeline import make_pipeline
X_train_tfidf = TfidfTransformer().fit_transform(bow_train4)
X_test_tfidf = TfidfTransformer().fit_transform(bow_test4)

In [189]:
logreg_cv.fit(X_train_tfidf, y_train_enc)
#printing the best parameters and accuracy
print("tuned hpyerparameters : (best parameters) ", logreg_cv.best_params_)
print(" Train data accuracy : ", logreg_cv.best_score_)

# printing accuracy of model on test data
print("Test data accuracy : ", logreg_cv.score(X_test_tfidf, y_test_enc))

tuned hpyerparameters : (best parameters)  {'C': 1}
 Train data accuracy :  0.6913310256310706
Test data accuracy :  0.6947334491341798


In [190]:
# finding or predicting target y values on test data
y_pred = logreg_cv.predict( X_test_tfidf)

# finding confusion matrix and classification report of the model's prediction based on actual and predicted values
from sklearn.metrics import classification_report, confusion_matrix
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y_pred ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y_pred ) )

# finding average f1 score of the model
from sklearn.metrics import f1_score
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y_pred ) ) )

 Confusion Matrix : 
 [[23253 11312]
 [ 5735 15543]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.80      0.67      0.73     34565
           1       0.58      0.73      0.65     21278

    accuracy                           0.69     55843
   macro avg       0.69      0.70      0.69     55843
weighted avg       0.72      0.69      0.70     55843

f1_score of logistic regression on scaled data : 0.646


We can notice that word analyzer with ngram range (1,2) gave us the best test accuracy

# Question 3

### Creating new X_train for further modifications with new features

In [0]:
#creating new dataframes
X_train_new = pd.DataFrame(df['body'])
X_test_new = pd.DataFrame(df1['body'])

In [0]:
# importing re to apply regular expressions
import re
# this line will find any html link present in the comment and keep count as 1 or 0 of not found
X_train_new['html_link'] = X_train_new['body'].apply(lambda x: 1 if(re.findall(r"http\S+", x)) else 0)
# same to check any hashtags
X_train_new['hashtag'] = X_train_new['body'].apply(lambda x: 1 if(re.findall(r'\B#\w*[a-zA-Z]+\w*', x)) else 0)
# this line will create length column which has value of each comments length
X_train_new['length'] = [len(i) for i in X_train_new['body']]

In [0]:
# the above functions are aplied also on test data
X_test_new['html_link'] = X_test_new['body'].apply(lambda x: 1 if(re.findall(r"http\S+", x)) else 0)
X_test_new['hashtag'] = X_test_new['body'].apply(lambda x: 1 if(re.findall(r'\B#\w*[a-zA-Z]+\w*', x)) else 0)
X_test_new['length'] = [len(i) for i in X_test_new['body']]

In [0]:
# creating a list of punctuations
punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
# creatin empty columns for punctuation count, capscount, and other character count if not punctuation
X_train_new['Punc-count'] = ''
X_train_new['CAPS-count'] = ''
X_train_new['Other-Chars'] = ''
for i in range(len(X_train_new)):
  # creating counter for each column
  punc_c = 0
  cap_c = 0
  other_c = 0
  tweet = X_train_new['body'][i]
  for word in tweet:
    for char in word:
      if char in punctuations:# checks if char is in punctuations
        punc_c += 1
      elif not char.isalpha() and char != '\n' and not char.isdigit() and char != ' ':#checks if char is not punctuation is not a new line or space or alphabet
        other_c += 1 
      if char.isupper(): # checks if char is in upper case or not
        cap_c += 1
  X_train_new['Punc-count'][i] = punc_c
  X_train_new['CAPS-count'][i] = cap_c
  X_train_new['Other-Chars'][i] = other_c

In [0]:
# creating same set of columns and precedure on test data as applyiied on train dataset
X_test_new['Punc-count'] = ''
X_test_new['CAPS-count'] = ''
X_test_new['Other-Chars'] = ''
for i in range(len(X_test_new)):
  punc_c = 0
  cap_c = 0
  other_c = 0
  tweet = X_test_new['body'][i]
  for word in tweet:
    for char in word:
      if char in punctuations:# checks if char is in punctuations
        punc_c += 1
      elif not char.isalpha() and char != '\n' and not char.isdigit() and char != ' ':#checks if char is not punctuation is not a new line or space or alphabet
        other_c += 1 
      if char.isupper(): # checks if char is in upper case or not
        cap_c += 1
  X_test_new['Punc-count'][i] = punc_c
  X_test_new['CAPS-count'][i] = cap_c
  X_test_new['Other-Chars'][i] = other_c

In [196]:
#printing shapes of data with new features
print(X_train_new.shape, X_test_new.shape)

(167529, 7) (55843, 7)


In [197]:
X_train_new.head()

,body,html_link,hashtag,length,Punc-count,CAPS-count,Other-Chars
0,I've always been taught it emerged from the ea...,0,0,125,2,2,0
1,"As an ECE, my first feeling as ""HEY THAT'S NOT...",0,0,229,10,17,2
2,Monday: Drug companies stock dives on good new...,0,0,61,2,2,0
3,i learned that all hybrids are unfertile i won...,1,0,139,11,0,0
4,Well i was wanting to get wasted tonight. Not...,0,0,84,4,2,0


In [0]:
# applying countvec with mindf 4 and ngram range 2
vect4 = CountVectorizer(min_df=4, stop_words='english', token_pattern=r"\b\w[\w’]+\b", ngram_range=(1, 2))
#bow_train4 = vect4.fit_transform(X_train_new)
#bow_test4 = vect4.transform(X_test_new)

In [199]:
# importing column transformer to apply countvec on new featutres dataset
from sklearn.compose import ColumnTransformer, make_column_transformer
# importing pioeline to apply each model
from sklearn.pipeline import make_pipeline
column_trans1 = make_column_transformer((vect4, 'body'), remainder='passthrough')
#column_trans1.fit_transform(X_train_new)
#from sklearn.decomposition import TruncatedSVD
#lsa = TruncatedSVD(n_components=100, random_state=1)
#from sklearn.preprocessing import MaxAbsScaler
#scaler = MaxAbsScaler()
#pipe = make_pipeline(column_trans1,scaler, lsa, logreg_cv)
pipe = make_pipeline(column_trans1, TfidfTransformer(), logreg_cv)
#fitting new data on all models
pipe.fit(X_train_new, y_train_enc)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('countvectorizer',
                                                  CountVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
      

In [0]:
# predicting test data
predicted = pipe.predict(X_test_new)

In [202]:
X_test_new.shape

(55843, 7)

In [0]:
y = pd.DataFrame(predicted)

In [205]:
print(" Confusion Matrix : \n", confusion_matrix( y_test_enc, y ) )
print('\n')
print("=== Classification Report ===")
print(classification_report( y_test_enc, y ) )

# finding average f1 score of the model
from sklearn.metrics import f1_score
print("f1_score of logistic regression on scaled data : {:.3f}".format( f1_score( y_test_enc, y ) ) )

 Confusion Matrix : 
 [[25819  8746]
 [ 8591 12687]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.75      0.75      0.75     34565
           1       0.59      0.60      0.59     21278

    accuracy                           0.69     55843
   macro avg       0.67      0.67      0.67     55843
weighted avg       0.69      0.69      0.69     55843

f1_score of logistic regression on scaled data : 0.594


### As we can see we have not got good f1 score with new features added, those features didnt help us

# Question 4
### implementing wordvec

In [0]:
# importing word_tokenize from nltk
from nltk.tokenize import word_tokenize

In [0]:
#creating empty list of tweets
tweet_lines_train = list()
# adding each tweet into a list
lines_train = df['body'].values.tolist()

In [208]:
print(pd.DataFrame(lines_train))

                                                        0
0       I've always been taught it emerged from the ea...
1       As an ECE, my first feeling as "HEY THAT'S NOT...
2       Monday: Drug companies stock dives on good new...
3       i learned that all hybrids are unfertile i won...
4       Well i was wanting to get wasted tonight.  Not...
...                                                   ...
167524  Eventually they're going to use crispr to make...
167525                  No wonder why its useless for me 
167526  Meditation brings wisdom, lack of meditation l...
167527                So india needs to make more saffron
167528  Is this system unique to primates? The title o...

[167529 rows x 1 columns]


In [0]:
#creating same list for test data
tweet_lines_test = list()
lines_test = df1['body'].values.tolist()

In [210]:
# downloading nltk punkt and stopwords packages
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# importing dowloaded stopwords of nltk library
from nltk.corpus import stopwords 
import string
for line in lines_train:
  tokens = word_tokenize(line) #tokenizing the words
  tokens = [w.lower() for w in tokens] # converting all words to lower case
  punc = str.maketrans('', '', string.punctuation) # converting punctuations to none
  stripped = [word.translate(punc) for word in tokens] # stripping words
  words = [ word for word in stripped if word.isalpha()] # checking if word is alphabets or not
  stop_words = set(stopwords.words('english')) # creating list of stopwords
  words = [ w for w in words if not w in stop_words] # creating words list after removing stop words
  tweet_lines_train.append(words) # appending those words to created list

In [0]:
# applying above same functions on test data
for line in lines_test:
  tokens = word_tokenize(line) #tokenizing the words
  tokens = [w.lower() for w in tokens]# converting all words to lower case
  punc = str.maketrans('', '', string.punctuation) # converting punctuations to none
  stripped = [w.translate(punc) for w in tokens]# stripping words
  words = [ word for word in stripped if word.isalpha()]# checking if word is alphabets or not
  stop_words = set(stopwords.words('english')) # creating list of stopwords
  words = [ w for w in words if not w in stop_words]# creating words list after removing stop words
  tweet_lines_test.append(words)# appending those words to created list

In [213]:
# printimg length of tweets of train set
len(tweet_lines_train)

167529

In [214]:
# printing length of tweets of test set
len(tweet_lines_test)

55843

In [215]:
# importing genism model to apply word2vec
import gensim
model = gensim.models.Word2Vec(sentences = tweet_lines_train, workers = 4, size = 100) # passing list of tweets that we created above to the model
words = list(model.wv.vocab)
print(len(words))

28449


In [37]:
model['good'] # printing vector representation of word good

array([ 0.09488058,  0.14240064,  0.9257356 , -0.2774913 , -1.0046504 ,
       -0.94951284, -0.29083833,  0.8891745 , -1.4607651 , -1.442384  ,
        1.2564552 , -0.9429087 , -2.3215823 , -0.23774625, -1.1593915 ,
       -2.2529566 , -2.1913712 , -0.35484153, -0.32041764,  0.8301951 ,
        1.0560063 ,  0.81379586,  0.82541084, -2.1180034 , -1.2237502 ,
       -1.1063329 ,  0.9364775 ,  0.5382029 , -0.05194172,  0.858803  ,
       -0.9182727 ,  0.8142993 , -0.37875992,  1.3758272 ,  0.26651007,
        2.018371  , -1.9237154 , -0.91978   ,  1.3341724 , -0.610091  ,
        0.8965078 , -1.6427361 , -1.2641482 ,  0.45386913,  2.1333842 ,
        0.25273982, -0.832319  ,  1.0900251 ,  0.6217188 ,  0.3691232 ,
       -0.67892855,  0.42648834, -1.8846406 , -1.549323  , -0.33330706,
        2.638092  , -0.27098605, -0.7238339 ,  0.1440763 , -0.28187224,
        1.2893581 ,  1.561416  ,  0.69859403, -0.524405  ,  1.8945317 ,
        0.16541326, -0.06328535,  0.5540277 ,  0.2964266 , -0.08

In [14]:
#model.wv.syn0

array([[ 3.2814380e-03, -8.8560528e-01, -1.7880061e-01, ...,
        -3.2024658e+00, -1.3733677e+00,  2.1517618e-01],
       [-1.4430740e+00,  1.3881652e+00, -2.3609892e-01, ...,
        -5.7840127e-01,  1.0842164e-01,  3.2258648e-01],
       [ 6.7940630e-02,  7.5197321e-01,  2.6384523e+00, ...,
        -1.2210764e+00, -4.6356502e-01,  1.0725466e+00],
       ...,
       [-4.2670019e-02,  4.9237370e-02,  1.0883017e-03, ...,
        -4.3781451e-03, -1.3166773e-02,  1.7828239e-02],
       [-5.6692525e-03,  3.0027037e-02,  9.2792949e-03, ...,
        -3.2877803e-02, -2.9108867e-02,  7.2778417e-03],
       [-5.8266740e-02,  3.8323585e-02,  2.3869639e-02, ...,
         2.1105003e-02, -2.1350157e-02, -3.6782838e-02]], dtype=float32)

In [60]:
model.wv.most_similar('trump') # printing words similar to trump

[('donald', 0.8760135173797607),
 ('president', 0.8714370727539062),
 ('supporters', 0.8669288158416748),
 ('hillary', 0.8490664958953857),
 ('republican', 0.8336113691329956),
 ('elected', 0.8259866833686829),
 ('clinton', 0.8180266618728638),
 ('voters', 0.7988657355308533),
 ('votes', 0.7851609587669373),
 ('voted', 0.7791755199432373)]

In [64]:
model.wv.most_similar('corona') # printing words similar to corona

[('bad', 0.7655016779899597),
 ('great', 0.6394116878509521),
 ('terrible', 0.5900849103927612),
 ('best', 0.5495654344558716),
 ('fantastic', 0.5202822685241699),
 ('decent', 0.4939470887184143),
 ('perfect', 0.49374642968177795),
 ('tough', 0.4932825267314911),
 ('matters', 0.4841466546058655),
 ('poor', 0.4780281186103821)]

In [59]:
# finding similarities between given 2 words
model.wv.similarity('good', 'best')

0.5495654

In [70]:
model.wv.similarity('virus', 'game')

-0.19189006

In [54]:
model.wv.similarity('trump', 'amazon')

0.17640732

In [55]:
model.wv.similarity('trump', 'president')

0.87143713

In [216]:
# creating same model for test data
model1 = gensim.models.Word2Vec(sentences = tweet_lines_test, workers = 4, size = 100)
words1 = list(model1.wv.vocab)
print(len(words1))

16352


In [0]:
#model1.wv.syn0

In [218]:
# finding similar words for the given word on test data
model1.wv.most_similar('good')

[('bad', 0.8871698379516602),
 ('great', 0.8120026588439941),
 ('terrible', 0.7062646150588989),
 ('whole', 0.6981465816497803),
 ('awesome', 0.6790273189544678),
 ('helps', 0.6772798299789429),
 ('enjoy', 0.6677002906799316),
 ('best', 0.6652629375457764),
 ('easy', 0.6638073921203613),
 ('lots', 0.6574831008911133)]

In [219]:
# finding similarity between 2 given words on test data
model1.wv.similarity('trump', 'president')

0.9523279

In [220]:
print(len(model.wv.syn0))

28449


In [0]:
# function to create document  vectors average also by removing words not present in model's vocabilary
def document_to_vector(list_words):
    #list_words = [word for word in doc if word in model.wv.vocab]
    return np.mean(model[list_words], axis=0)

In [0]:
# creating sample dataframe to apply the above function on train data
sample =pd.DataFrame(df['body'])

In [0]:
# creating new column which has words splitted of each comment
sample['word_list']= [i.lower().split() for i in sample['body']]

In [224]:
# printing dataset
sample.head()

,body,word_list
0,I've always been taught it emerged from the ea...,"[i've, always, been, taught, it, emerged, from..."
1,"As an ECE, my first feeling as ""HEY THAT'S NOT...","[as, an, ece,, my, first, feeling, as, ""hey, t..."
2,Monday: Drug companies stock dives on good new...,"[monday:, drug, companies, stock, dives, on, g..."
3,i learned that all hybrids are unfertile i won...,"[i, learned, that, all, hybrids, are, unfertil..."
4,Well i was wanting to get wasted tonight. Not...,"[well, i, was, wanting, to, get, wasted, tonig..."


In [0]:
# importing stopwords to remove it from columns of list of words from sample data
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
sample['word_list']=sample['word_list'].apply(lambda x: [item for item in x if item not in stop_words])

In [226]:
# printing dataframe after adding new column
sample.head()

,body,word_list
0,I've always been taught it emerged from the ea...,"[i've, always, taught, emerged, earth, impace...."
1,"As an ECE, my first feeling as ""HEY THAT'S NOT...","[ece,, first, feeling, ""hey, that's, not-"", th..."
2,Monday: Drug companies stock dives on good new...,"[monday:, drug, companies, stock, dives, good,..."
3,i learned that all hybrids are unfertile i won...,"[learned, hybrids, unfertile, wont, read, clic..."
4,Well i was wanting to get wasted tonight. Not...,"[well, wanting, get, wasted, tonight., much, r..."


In [230]:
# creating new column which would have avg vector value of each comment
sample['doc_vector'] = sample.word_list.apply(document_vector)

KeyError: ignored

In [0]:
# installing spacy and other packages to use word2vec from spacy library
#!pip install spacy
#!python -m spacy download en_core_web_lg
#!python -m spacy download en

In [134]:
'''import spacy
#import en_core_web_lg
nlp = spacy.load("en_core_web_lg")
docs_train = [nlp(d.decode()).vector for d in X_train]
X_train = np.vstack(docs_train)
X_train.shape'''

'import spacy\n#import en_core_web_lg\nnlp = spacy.load("en_core_web_lg")\ndocs_train = [nlp(d.decode()).vector for d in X_train]\nX_train = np.vstack(docs_train)\nX_train.shape'